# Суммаризация длинного текста используя `Map reduce` подход

In [4]:
%pip install -q langchain langchain-gigachat transformers sentence-transformers

In [32]:
import os
from pathlib import Path

from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_gigachat import GigaChat
from langchain_text_splitters import RecursiveCharacterTextSplitter

from google.colab import userdata

In [11]:
filepath = "/content/long_texts/meeting_transcription.txt"

In [12]:
with open(filepath, "r", encoding="utf-8") as file:
  text = file.read()

len(text)

85234

In [13]:
llm = GigaChat(
    credentials=userdata.get("GIGACHAT_APIKEY"),
    scope=userdata.get("GIGACHAT_SCOPE"),
    model="gigachat:latest",
    profanity_check=False,
    verify_ssl_certs=False,
)

In [14]:
llm.invoke("Какие лучшие практики для суммаризации длинных текстов?")

AIMessage(content='Для эффективной суммаризации больших и сложных текстов используют следующие проверенные практики и методы:\n\n### 1. **Анализ заголовков и структуры текста**\n   - Просмотрите оглавление (если есть), заголовки разделов и подразделов.\n   - Извлечённые заголовки помогут сразу определить основные темы текста.\n\n### 2. **Определение ключевых идей**\n   - Выделите ключевые идеи каждой главы или раздела.\n   - Это наиболее важные утверждения автора, отражающие основную мысль текста.\n\n### 3. **Построение карты мыслей (mind map)**\n   - Представьте структуру текста визуально — нарисуйте графическое отображение основных тем и связей между ними.\n   - Используйте диаграммы Венна, деревья решений, кластеры и другие инструменты визуализации для упрощения понимания взаимосвязей.\n\n### 4. **Чтение аннотаций и выводов**\n   - Введение и заключение содержат выводы автора, подводящие итог всему тексту.\n   - Часто аннотация или резюме текста уже содержит сжатый вариант всей осно

In [20]:
# Расчёт количества токенов

tokens_count = llm.tokens_count([text])[0].tokens

In [19]:
tokens_count

23094

In [22]:
# Расчёт приблизательного количества символов в одном токене

len(text) / tokens_count

3.6907421841170867

In [26]:
# Расчёт приблизительного размера контекстного окна модели

3.6 * 8000

28800.0

In [28]:
chunk_size = 24000

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=100, length_function=len
)

chunks = splitter.split_text(text)

len(chunks)

4

In [29]:
prompt = """Ты - профессиональный ассистент для суммаризации аудио транскрипций. Твоя задача - создать четкую, структурированную и информативную summary аудиозаписи.

**ИСХОДНЫЙ ТЕКСТ:**
{text}

**ИНСТРУКЦИИ ДЛЯ SUMMARY:**

1. **ТИП SUMMARY:** Создай конспект в формате "ключевые идеи + выводы"
2. **СТРУКТУРА:**
   - **Основная тема:** 1-2 предложения
   - **Ключевые моменты:** 3-5 самых важных идей
   - **Конкретные примеры/факты:** 2-3 значимых детали
   - **Выводы/рекомендации:** что следует запомнить или сделать
   - **Общий тон/настроение:** нейтральный, информативный, критический и т.д.

3. **ОСОБЕННОСТИ АУДИО ТРАНСКРИПЦИЙ:**
   - Игнорируй слова-паразиты, повторы, паузы ("э-э-э", "короче", "типа")
   - Сохрани важные эмоциональные акценты, если они есть
   - Учитывай, что это устная речь - могут быть незаконченные мысли

4. **СТИЛЬ:** Профессионально, но доступно. Используй маркированные списки для наглядности.

**РЕЗУЛЬТАТ ДОЛЖЕН БЫТЬ ПОЛЕЗЕН ДЛЯ ТОГО, КТО НЕ СЛУШАЛ АУДИОЗАПИСЬ.**
"""

In [33]:
chain = ChatPromptTemplate.from_template(prompt) | llm | StrOutputParser()

summaries = []
for chunk in tqdm(chunks):
  summary = chain.invoke({"text": chunk})
  summaries.append(summary)

100%|██████████| 4/4 [00:10<00:00,  2.74s/it]


In [34]:
summary = "\n\n".join(summaries)

In [35]:
MEETING_PROTOCOL_PROMPT = """Задача: На основе транскрибации (текстовой расшифровки) совещания составить официальный протокол в структурированном формате.

## Шаг 1. Вводные данные
 * Название компании: [вписать]

 * Вид документа: [например, "Протокол производственного совещания"]

 * Дата проведения: [дд.мм.гггг]

 * Место проведения: [если важно]

## Шаг 2. Участники
 * Председатель: [ФИО, способ участия (лично/онлайн)]

 * Секретарь: [ФИО]

 * Присутствовали:

 * * [ФИО] – лично/онлайн

 * * [ФИО] – лично/онлайн
(и т. д.)

## Шаг 3. Повестка дня
Перечислить основные вопросы, которые обсуждались:

1. [Тема 1]

2. [Тема 2]

3. [Прочие вопросы]

## Шаг 4. Обсуждение (СЛУШАЛИ)
Кратко изложить ключевые моменты выступлений:

 * [ФИО или "Всех присутствующих"]: [основные тезисы, мнения, предложения]

## Шаг 5. Решения (ПОСТАНОВИЛИ)
Оформить в виде таблицы (если нужно) или списка:

| № п/п | Содержание поручения | Ответственный исполнитель (ФИО) | Срок выполнения |
| 1.1 | [Суть задачи] |	[ФИО] |	[дд.мм.гггг] |
| 1.2 |	[Суть задачи] |	[ФИО] |	[дд.мм.гггг] |
Или списком:

[Задача] – [Исполнитель], срок до [дата].

## Шаг 6. Подписи
Председатель: __________ / [ФИО]

Секретарь: __________ / [ФИО]

Дополнительные указания:
Сохранять официально-деловой стиль.

Избегать лишних деталей, оставлять только ключевые решения.

Если в обсуждении были разногласия – указать альтернативные мнения.

Даты и сроки проверять на актуальность.

Пример заполнения:
"1.1 Восстановить документы по проекту 'Алькор' – Кравцова Л.М., срок до 10.06.2025"

Формат вывода: Готовый протокол в формате, аналогичном образцу.


Транскрибация совещания:
{summary}
"""

In [36]:
chain = ChatPromptTemplate.from_template(MEETING_PROTOCOL_PROMPT) | llm | StrOutputParser()

meeting_protocol = chain.invoke({"summary": summary})

with open("Протокол_совещания.txt", "w", encoding="utf-8") as file:
  file.write(meeting_protocol)